In [ ]:
from flow.core.params import VehicleParams
from flow.controllers import IDMController, ContinuousRouter
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams

# Example
from custom_network import myNetwork

# Ours
from custom_network import CustomBottleneckNetwork

In [ ]:
ADDITIONAL_NET_PARAMS = {
    "speed_limit": 30,
}

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)

In [ ]:
vehicles = VehicleParams()
vehicles.add(veh_id="human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=12)

sim_params = SumoParams(sim_step=0.1, render=True)

initial_config = InitialConfig(bunching=40)

In [ ]:
# These are environment params, different from Net_params

from flow.envs.ring.accel import AccelEnv, ADDITIONAL_ENV_PARAMS
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

In [ ]:
from flow.core.experiment import Experiment

flow_params = dict(
    exp_tag='test_network',
    env_name=AccelEnv,
    network=CustomBottleneckNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
)

# number of time steps
flow_params['env'].horizon = 1500
exp = Experiment(flow_params)

# run 
_ = exp.run(1)

## Example

In [ ]:
ADDITIONAL_NET_PARAMS = {
    "radius": 40,
    "num_lanes": 1,
    "speed_limit": 30,
}

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)

In [ ]:
vehicles = VehicleParams()
vehicles.add(veh_id="human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=22)

sim_params = SumoParams(sim_step=0.1, render=True)

initial_config = InitialConfig(bunching=40)

In [ ]:
from flow.envs.ring.accel import AccelEnv, ADDITIONAL_ENV_PARAMS

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

In [ ]:
from flow.core.experiment import Experiment

flow_params = dict(
    exp_tag='test_network',
    env_name=AccelEnv,
    network=myNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
)

# number of time steps
flow_params['env'].horizon = 1500
exp = Experiment(flow_params)

# run 
_ = exp.run(1)